<a href="https://colab.research.google.com/github/moung1012/Numpy-Pandas/blob/master/keras5_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 합성곱 신경망 소개
- 컨브넷(convnet)이라 불리우는 합성곱 신경망(convolutional neural network)소개
- 이 딥러닝 모델은 대부분 컴퓨터 비전 애플리케이션에 사용
- 작은 훈련 데이터셋을 사용한 이미지 분류 문제에 컨브넷 적용하는 사례


In [ ]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
#컨브넷은 (image_height, image_width, iamage_channels) 크기의 입력 텐서 사용
#이 예제는 MNIST 이미지 포맷인 (28, 28, 1)크기의 입력을 처리하도록 컨브넷을 설정해야 한다
#이 때문에 매개변수로 input_shape=(28, 28, 1)전달

In [ ]:
model.summary()

- Conv2D와 MaxPooling2D 층의 출력은 (height, width, channels)크기의 3D 텐서
- 높이와 넓이 차원은 네트워크가 깊어 질수록 작아지는 경향이 있다
- 채널의 수는 Conv2D 층에 전달된 첫 번째 매개변수에 의해 조절(32개 또는 64개)
- 다음 단계에서 마지막 층의(3, 3, 64)출력 텐서를 완전 연결 네트워크에 주입
- 이 네트워크는 Dense층을 쌓는 분류기: 이 분류기는 1D 벡터를 처리하는데 이전 층의 출력이 3D 텐서이다
- 먼저 3D 출력을 1D 텐서로 펼쳐야 한다.
- 그 다음 Dense층을 추가



In [ ]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))
# 10개의 클래스를 분류하기 위해 마지막 층을 출력 크기를 10으로 하고 소프트맥스 활성화 함수를 사용한다

In [ ]:
model.summary()


(3, 3, 64)의 3D 텐서의  출력이 (576,)의 1D 벡터 크기로 펼처진후 Dense층으로 주입  
flatten (Flatten)            (None, 576)               0           
MNIST 숫자 이미지에 이 컨브넷을 훈련


In [ ]:
from keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [ ]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=5, batch_size=64)

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

In [ ]:
test_acc

기존의 완전 연결 네트워크보다 더 높은 테스트 정확도를 컴브넷이 얻었다  
Conv2D와 MaxPolling2D층이 어떤 일을 하는지 살펴보자

---
### 합성곱 연산
- 완전 연결 층과 합성곱 층 사이의 근본적인 차이
- Dense 층은 입력 특성 공간에 있는 전역 패턴(MNIST 숫자 이미지에서는 모든 픽셀에 거친 패턴)을 학습
- 합성곱 층은 지역 패턴을 학습한다
  - 이미지일 경우 작은 2D 윈도우로 입력에서 패턴을 찾는다(앞의 예시는 3 X 3크기)
---

**이 핵심 특징은 컨브넷의 두 가지 흥미로운 성질을 제공**
1. 학습된 패턴은 평행 이동 불변성(translation invariant)을 가진다
  - 컨브넷이 이미지의 오른쪽 아래 모서리에서 어떤 패턴을 학습했다면 다른 곳(EX: 왼쪽 위 모서리)에서도 이 패턴을 인식할 수 있다
  - 완전 연결 네트워크는 새로운 위치에 나타난 것은 새로운 패턴으로 학습해야 한다
  - 이 성질로 인해 적은 수의 훈련 샘플을 사용하더라도 일반화 능력을 가진 표현을 학습할 수 있다
2. 컨브넷은 패턴의 공간적 계층 구조를 학습할 수 있다
  - 첫 번째 합성곱 층이 에지 같은 작은 지역 패턴을 학습
  - 두 번째 합성곱 층은 첫 번째 층의 특성으로 구성된 더 큰 패턴을 학습하는 식이다
  - 이런 방식을 사용해 컨브넷은 매우 복잡하고 추상적인 시각적 개념을 효과적으로 학습할 수 있다
  


---
합성곱 연산은 **특성 맵(Feature map)** 이라고 부르는 3D 텐서에 적용된다  
**특성 맵**  
- 이 텐서는 2개의 공간 축(height, width)와 깊이 축(channel 